In [1]:
import dmd_with_power_control_single_task as DMD
import imageio
import numpy as np
import time
from glob import glob
import os
import importlib
importlib.reload(DMD)
import napari as nap

In [2]:
XP_FOLDER = r'D:\Data\MBONS\G1' # 
TARGET_FOLDER = XP_FOLDER + r'\1_target_20241126_125147' # D:\Data\\
GENERAL_PARAMETERS = r'C:\\Users\\Light Sheet User\\Desktop\\DMD_parameters.pickled'

mirrors = DMD.dmd(XP_FOLDER, TARGET_FOLDER, GENERAL_PARAMETERS)
# Prints basic information about the DMD.
mirrors.idle()
mirrors.select_ROIs()

Loading library: C:/Program Files/ALP-4.3/ALP-4.3 API/x64/alp4395.dll
DMD found, resolution = 1920 x 1200.
DMD patterns folder already exists.


c:\Users\Light Sheet User\Documents\GitHub\DMD\dmd_with_power_control_single_task.py:43: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\\\Users\\\\Light Sheet User\\\\Desktop\\\\DMD_parameters.pickled'>
  self.parameters = pickle.load(open(general_parameters_folder, 'rb'))


In [3]:
mirrors.import_N_rois(exp='1_2')

Found 3 ROI(s)


DELAY (0dt) CONDITIONING

In [5]:
masks_path = r'D:\Data\MBONS\G1\excitation_patterns\1_2'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\1_3_20241126_135140' #D:\Data\MBONS\G1\
protocol_name = r'\1_3'
images_folder = mirrors.xp_folder + current_acquisition


img2 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img1 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

steps = []

power = .08 # changed from 2

on_time = 10
off_time = 30
train_off_time = 10
rep_dead_time = 20

steps.append(time.time())
for i in range(3):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)

time.sleep(30)

steps.append(time.time())
for i in range(10):
    mirrors.project(odpun, power); time.sleep(on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(rep_dead_time)

time.sleep(40)

steps.append(time.time())
for i in range(3):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)


# SAVE THE PROTOCOL



filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

timesteps=[]
for t in steps:
    for num, volume in enumerate(all_volumes):
        current = os.path.getctime(volume)
        if current - t > 0:
            timesteps.append(num)
            break

np.savez(images_folder + protocol_name,\
         TM=timesteps, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", 
                         "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", 
                         "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         ], \
                durations=[on_time, off_time, on_time, off_time, 
                           on_time, off_time, on_time, off_time,
                           on_time, off_time,on_time, 60,
                           on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                           on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time,on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time,on_time, train_off_time, on_time, 60,
                            on_time, off_time, on_time, off_time, 
                           on_time, off_time, on_time, off_time,
                           on_time, off_time,on_time, off_time,
                           ]
                ) # 17 mins 31 sec 

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_17296\2593680140.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_17296\2593680140.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_17296\2593680140.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input voltage 0.080000 

DMD ON,  input

In [10]:
# fast single odour in training

masks_path = r'D:\Data\g12yfp\excitation_patterns\5_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\5_5_20240718_120207' #D:\Data\g12yfp\
protocol_name = r"/5_5"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

on_time = 10
off_time = 30
train_off_time = 10
rep_dead_time = 10

for i in range(2):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(60)

for i in range(9):
    mirrors.project(odpun, power); time.sleep(on_time); mirrors.idle(); time.sleep(train_off_time)

mirrors.project(odpun, power); time.sleep(on_time); mirrors.idle(); time.sleep(60)

for i in range(3):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "1", "dark",  "1", "dark", 
                         "odpun", "dark", "odpun", "dark", "odpun", "dark", "odpun", "dark", "odpun", "dark", 
                         "odpun", "dark", "odpun", "dark", "odpun", "dark", "odpun", "dark", "odpun", "dark",
                         "1", "dark", "1", "dark", "1", "dark", 
                         ], \
                durations=[on_time, off_time, on_time, off_time, on_time, 60,
                           on_time, train_off_time, on_time, train_off_time,on_time, train_off_time,
                             on_time, train_off_time,on_time, train_off_time,on_time, train_off_time,
                               on_time, train_off_time,on_time, train_off_time,on_time, train_off_time,
                                 on_time, 60,
                            on_time, off_time, on_time, off_time, on_time, off_time
                           ]
                ) # 13

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_14548\1096042012.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_14548\1096042012.py:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_14548\1096042012.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 



SAM's delay protocol

In [5]:
masks_path = r'D:\Data\g12yfp\excitation_patterns\7_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\7_2_20240724_125525' # D:\Data\g12yfp\
protocol_name = r"/7_2"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2.2 # changed from 2on_time = 20
off_time = 40
train_off_time = 10 # changed from 10
rep_dead_time = 20

for i in range(2):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(60)

for i in range(10):
    mirrors.project(img1, power); time.sleep(20); 
    mirrors.project(odpun, power); time.sleep(10); mirrors.idle(); time.sleep(15)
    mirrors.project(img2, power); time.sleep(30); mirrors.idle(); time.sleep(15)

time.sleep(105)

for i in range(2):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(60)

# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark",  "2", "dark", "1", "dark",  "2", "dark", "1", "dark",  "2", "dark",  
                        "1", "odpun", "dark", "2", "dark","1", "odpun", "dark", "2", "dark","1", "odpun", "dark", "2", "dark",
                        "1", "odpun", "dark", "2", "dark","1", "odpun", "dark", "2", "dark","1", "odpun", "dark", "2", "dark",
                        "1", "odpun", "dark", "2", "dark","1", "odpun", "dark", "2", "dark","1", "odpun", "dark", "2", "dark",
                        "1", "odpun", "dark", "2", "dark",
                         "1", "dark",  "2", "dark", "1", "dark",  "2", "dark", "1", "dark",  "2", "dark",  
                         ], \
                durations=[on_time, off_time, on_time, off_time, on_time, 60, 
                           20, 10, 15, 30, 15, 20, 10, 15, 30, 15, 20, 10, 15, 30, 15, 
                           20, 10, 15, 30, 15, 20, 10, 15, 30, 15, 20, 10, 15, 30, 15, 
                           20, 10, 15, 30, 15, 20, 10, 15, 30, 15, 20, 10, 15, 30, 15, 
                           20, 10, 15, 30, 120,
                           on_time, off_time, on_time, off_time, on_time, 60
                           ]
                ) # 22 mins 51 sec

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_16244\3352302025.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_16244\3352302025.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_16244\3352302025.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input voltage 2.200000 

DMD ON,  input